# Домашнее задание 3 - Metric Learning
<br>

Задание выполнил(а): *(впишите свои фамилию и имя)*
<br>

**Дедлайн:** 08.12.2020 23:59


### О задании
В этом домашнем задании вы познакомитесь с возможностями SAS для реализации Metric Learning, обучите нейросеть в парадигме Metric Learning и используете ее результаты для классификации изображений.
<br>

**Задание построено следующим образом:**
- Задание 1: Загрузка данных в CAS (1 балл)
- Задание 2: Построение модели (2 балла)
- Задание 3: Oбучение модели и проверка качества (2 балла)
- Задание 4: Сохранение модели (1 балл)
- Задание 5: Загрузка сохраненной модели и скоринг обучающей и тестовой выборок (3 балла)
- Задание 6: Обучите KNeighborsClassifier на основе проскоренных данных (3 балла)
- Задание 7: Визуализация выходов нейросети (3 балла)

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from swat import *
from dlpy import Model, Sequential
from dlpy.layers import *
from dlpy.images import ImageTable
from swat import *
from dlpy.images import *
from dlpy.image_embedding import *
from dlpy.embedding_model import EmbeddingModel

import swat
swat.options.cas.dataset.max_rows_fetched = 1000000

**Подключение к серверу CAS**<br>
Подключитесь к серверу CAS под своим логином и паролем.<br>
Загрузите следующие Action Sets: 'image', 'deepLearn' и 'astore'.<br>

In [ ]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = r"/tmp/4fix/trustedcerts.pem"

# Создаем новую сессию CAS:

# Ваш код здесь

# Загружаем action sets:

# Ваш код здесь

**Задание 1: Загрузка данных в CAS**<br>
Для обучения сетей Metric Learning используется отдельный класс CAS-таблиц - ImageEmbeddingTable. Прочитайте документацию к этому классу, загрузите обучающую выборку в CAS. Архитектура, которую мы будем использовать - triplet, количество наблюдений задайте в соответствии с количеством картинок в обучающей выборке. Проверьте, что данные загрузились и таблица не пустая. Выведите количество троек в полученной таблице с различными наборами классов.

In [ ]:
train_dir = #Путь к директории с обучающими данными
train_table = ImageEmbeddingTable.load_files(s,

In [ ]:
# Проверьте что данные загрузились

In [ ]:
# Выведите количество троек с различными наборами классов


**Задание 2: Построение модели**<br>
Постройте Sequential-модель, соответствующую изображенной на картинке, c помощью dlpy.
![model](model.png)
Создайте отдельно создайте полносвязный Embedding-слой размерностью 2 с функцией активации identity. Ознакомившись с докуметацией, создайте модель для Metric Learning с помощью метода build_embedding_model класса EmbeddingModel. Используйте архитектуру triplet, параметру margin задайте значение -3.0.

In [ ]:
# Постройте Sequential-модель

In [ ]:
# Cоздайте полносвязный Embedding-слой
embedding_layer = Dense(

In [ ]:
# Cоздайте модель для Metric Learning
model_tr = EmbeddingModel.build_embedding_model(

**Задание 3: Oбучение модели и проверка качества**<br>
а) Обучите построенную модель. Гиперпараметры для обучения уже заданы, менять их не требуется.

In [ ]:
from dlpy.model import Optimizer, MomentumSolver, AdamSolver
from dlpy.lr_scheduler import *

lr = 0.01

solver = AdamSolver(lr_scheduler=StepLR(learning_rate=lr, step_size=30), clip_grad_max = 100, clip_grad_min = -100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=32, log_level=2, max_epochs=100, reg_l2=0.0001)

In [ ]:
train_history = model_tr.fit_embedding_model(data=train_table, optimizer=optimizer, n_threads=4, seed=1234,record_seed=23435, train_from_scratch=True)

б) Проверьте качество модели на обучающей выборке спомощью функции predict.

In [ ]:
model_tr.predict(

в) Выполните код в ячейке ниже - он подсчитает статистики по двум переменным - '\_DL\_Pred0\_' и '\_DL\_Pred1\_'. <br>

Первая показывает полученное с помощью модели расстояние между первой картинкой в тройке (anchor) и второй картинкой в тройке, принадлежащей тому же классу, что и anchor. Переменная '\_DL\_Pred1\_' показывает расстояние между anchor и третьей картинкой в тройке, принадлежащей к другому классу, отличному от класса anchor-картинки. <br>

Логично, что если модель хорошо обучилась, расстояние между картинками одного класса должно быть меньше чем между картинками разных классов. Проверьте, для какого процента наблюдений в обучающей выборке выполняется неравенство \_DL\_Pred0\_ < \_DL\_Pred1\_.

In [ ]:
s.summary(table=model_tr.valid_res_tbl, inputs=['_DL_Pred0_', '_DL_Pred1_'])

In [ ]:
# Проверьте, для какого процента наблюдений в обучающей выборке выполняется неравенство _DL_Pred0_ < _DL_Pred1_


**Задание 4: Сохранение модели**
Сохраните модель при помощи метода deploy_embedding_model класса EmbeddingModel в формате astore. Не нужно сохранять все три ветки модели - сохраните только одну.

In [ ]:
branch_model = model_tr.deploy_embedding_model(

**Задание 5: Загрузка сохраненной модели и скоринг обучающей и тестовой выборок**<br>
а) Дополните код ниже, чтобы загрузить сохраненную модель. Затем загрузите две обычные ImageTable - с обучающей и тестовой выборкой.

In [ ]:
astore_path = #Путь к сохраненной в формате astore модель
with open(astore_path, mode='rb') as file: 
    fileContent = file.read()    
store_ = blob(fileContent)
s.astore.upload(

In [ ]:
train_data = 

test_data = 

б) Проскорьте обучающие и тестовые данные с помощью action score. Сохраните результаты скоринга в два отдельных csv-файла.

In [ ]:
#Скоринг обучающих данных
s.score(

In [ ]:
#Скоринг тестовых данных
s.score(

**Задание 6: Обучите KNeighborsClassifier на основе проскоренных данных** <br>
а) Загрузите сохраненные csv-файлы с помощью pandas. Удалите из наборов данных лишние переменные (с постфиксом \_fake\_output\_). Выходы модели хранятся в переменных '\_LayerAct\_7_0\_0\_0\_' и '\_LayerAct\_7\_0\_0\_1\_'. Для обоих наборов данных сохраните в отдельные переменные признаки (X) и целевую переменную(y). 

In [ ]:
# Загрузка данных...

In [ ]:
X_train = 
y_train = 

X_test = 
y_test = 

б) Обучите KNeighborsClassifier из sklearn.neighbors на основе проскоренных нейросетевой моделью обучающих данных. Подберите количество ближайших соседей самостоятельно, чтобы получить как можно лучшее качество на тестовой выборке. Сравните получившееся качество классификации по accuracy_score с моделью, которую вы обучили в первом домашнем задании. Как вы думаете, от какого параметра в заданной нами нейросети в наибольшей степени зависит полученное качество?

In [ ]:
# Создайте и обучите классификатор
from sklearn.neighbors import KNeighborsClassifier
knn = 

In [ ]:
# Проверьте качество модели
from sklearn.metrics import accuracy_score


**Задание 7: Визуализация выходов нейросети** <br>
Постройте визуализации выходов модели с помощью matplotlib на обучающих и тестовых данных, аналогичную представленной на картинке.
![vis](vis.PNG)

In [ ]:
# Ваш код здесь